In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [3]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="gpt-oss:20b", temperature=0)

In [4]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

system_prompt = """You are a travel agent. 
IMPORTANT: When using the search-flight tool, you MUST use the following rules:
1. Dates must be in 'dd/mm/yyyy' format (e.g., 31/03/2026).
2. All dates must be in the future.
3. If the user asks for a one-way flight, ensure you still provide the correct parameters required by the tool.
No follow up questions."""

agent = create_agent(
    model=model,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt=system_prompt
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="""Create an effective prompt template for getting flights""")]},
    config
    )

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Create an effective prompt template for getting flights', additional_kwargs={}, response_metadata={}, id='89d4ef99-5a0b-4d7a-a60f-5fd2aa540987'),
              AIMessage(content='Here\'s an effective prompt template for flight searches:\n\n"Find [number] flights from [departure city/airport] to [destination city/airport] on [departure date in dd/mm/yyyy format]. [Optional: Include return date if round-trip]. [Specify cabin class preference: economy, premium, business, or first class]. [Optional: Sort by price, duration, or quality]. [Optional: Allow date flexibility of ±X days]. [Specify currency for pricing]."\n\nExample:\n"Find 5 flights from Paris CDG to Tokyo NRT departing 15/03/2026. Sort by price, allow ±2 days flexibility, economy class, display in EUR."\n\nThis template ensures all required parameters are included while allowing customization. The system will automatically handle location resolution, date validation, and result grouping by pr

In [7]:
print(response["messages"][-1].content)

Here's an effective prompt template for flight searches:

"Find [number] flights from [departure city/airport] to [destination city/airport] on [departure date in dd/mm/yyyy format]. [Optional: Include return date if round-trip]. [Specify cabin class preference: economy, premium, business, or first class]. [Optional: Sort by price, duration, or quality]. [Optional: Allow date flexibility of ±X days]. [Specify currency for pricing]."

Example:
"Find 5 flights from Paris CDG to Tokyo NRT departing 15/03/2026. Sort by price, allow ±2 days flexibility, economy class, display in EUR."

This template ensures all required parameters are included while allowing customization. The system will automatically handle location resolution, date validation, and result grouping by price/duration.


In [8]:
prompt = """Find 5 flights from London LHR to Tokyo NRT departing 15/03/2026. Sort 
            by price, allow ±2 days flexibility, economy class, display in GBP."""


response = await agent.ainvoke(
    {"messages": [HumanMessage(content=prompt)]},
    config
    )


pprint(response)

Failed to send compressed multipart ingest: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. Please confirm your internet connection. ConnectTimeout(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/multipart (Caused by ConnectTimeoutError(<HTTPSConnection(host='api.smith.langchain.com', port=443) at 0x78be68135280>, 'Connection to api.smith.langchain.com timed out. (connect timeout=10.0)'))"))
Content-Length: 4980
API Key: lsv2_********************************************betrace=019c9b7d-a4b7-7d41-91b9-dc40275906e9,id=019c9b7d-a5be-7373-9385-58ca8c496df8; trace=019c9b7d-a4b7-7d41-91b9-dc40275906e9,id=019c9b7d-a5bd-7363-8036-1879a314a460; trace=019c9b7d-a4b7-7d41-91b9-dc40275906e9,id=019c9b7d-a4b7-7d41-91b9-dc40275906e9; trace=019c9b87-b42a-77b1-a186-62cb2fc62d33,id=019c9b87-b42a-77b1-a186-62cb2fc62d33; trace=019c9b87-b42a-77b1-a186-62cb2fc62d33,id=019c9b87-b444-7c90

In [ ]:
print(response["messages"][-1].content)